In [ ]:
import google.colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if IN_COLAB:
    sys.path.append('/content/drive/MyDrive/Lid_Driven_Cavity')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from Pressure_Poisson_Equation_Solver import setup_solvers
from Quasi_Rhie_Chow import quasi_Rhie_Chow_iteration, converged
from Boundary_Conditions import update_ghost_cell_pressure, update_ghost_cell_velocities

In [ ]:
# Constants

# Acceleration due to gravity
g   = 0 # 9.8   # m/s2

# Density of Fluid
rho = 1.0   # kg/m3

# Kinematic Viscosity
nu  = 0.01

In [ ]:
# Grid Generation
M = 100

Dx = Dy = 1.0 / M

# M x M cells + Ghost cells on either boundaries
x = np.linspace(-Dx/2, 1 + Dx/2, M+2)
y = np.linspace(-Dy/2, 1 + Dy/2, M+2)

In [ ]:
# Time Step
Dt = 0.001

# CFL Number
num_CFL = Dt / Dx
print('Courant Number : ', num_CFL)

Courant Number :  0.1


In [ ]:
u = np.zeros((1, M+2, M+2))
v = np.zeros((1, M+2, M+2))

p = np.zeros((1, M+2, M+2))
p[0, 1:-1, 1:-1] = - rho * g * (y[1:-1] - y[-2])

update_ghost_cell_velocities(u[0], v[0])
update_ghost_cell_pressure(p[0], rho, g, Dx, Dy)

In [ ]:
# Setting up pressure poisson eqn solver
setup_solvers(M, Dx, Dy)

In [ ]:
u_n = np.copy(u[0])
v_n = np.copy(v[0])
p_n = np.copy(p[0])

u_np1, v_np1, p_np1 = quasi_Rhie_Chow_iteration(
    u_n, v_n,
    p_n,
    rho, nu, g,
    Dt,
    Dx, Dy
)

u = np.append(u, [u_np1], axis=0)
v = np.append(v, [v_np1], axis=0)
p = np.append(p, [p_np1], axis=0)

while not converged(u_n, v_n, p_n, u_np1, v_np1, p_np1) :

    u_n = np.copy(u_np1)
    v_n = np.copy(v_np1)
    p_n = np.copy(p_np1)

    u_np1, v_np1, p_np1 = quasi_Rhie_Chow_iteration(
        u_n, v_n,
        p_n,
        rho, nu, g,
        Dt,
        Dx, Dy
    )

    u = np.append(u, [u_np1], axis=0)
    v = np.append(v, [v_np1], axis=0)
    p = np.append(p, [p_np1], axis=0)



In [ ]:
solution_num = 'Solution_5'

np.save(solution_num + '_U', u)
np.save(solution_num + '_V', v)
np.save(solution_num + '_p', p)


In [ ]:
if IN_COLAB:
    !cp Solution_5_U.npy "/content/drive/MyDrive/Lid_Driven_Cavity"
    !cp Solution_5_V.npy "/content/drive/MyDrive/Lid_Driven_Cavity"
    !cp Solution_5_p.npy "/content/drive/MyDrive/Lid_Driven_Cavity"